# Explore and cluster the neighborhoods in Toronto

## Part 1

### Code to scrape the Wikipedia page in order to obtain the data into a pandas dataframe

#### Installing Libraries

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
 pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [4]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [5]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



#### Importing Libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import json 
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("All Packages Imported")

All Packages Imported


#### Downloading Data from Webpage

In [7]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [8]:
soup = BeautifulSoup(html, 'lxml')
type(soup)

bs4.BeautifulSoup

#### Creating a Datafame

In [9]:
# instantiate the dataframe
rows = soup.find_all('tr')
row_1=rows[:1]
#print(row_1)
for row in row_1:
    row_td = row.find_all('th')
#print(row_td)
type(row_td)
str_cells = str(row_td)
cleantext = BeautifulSoup(str_cells, "lxml").get_text()
cleantext=cleantext[1:-2]
column_names=cleantext.split(', ')
toronto = pd.DataFrame(columns=column_names)
toronto

,Postcode,Borough,Neighbourhood


In [10]:
# Appending the Data
rows = soup.find_all('tr')
for n in range(1,289):    
    row_1=rows[n:n+1]
    #print(row_1)
    for row in row_1:
        row_td = row.find_all('td')
    #print(row_td)
    type(row_td)
    str_cells = str(row_td)
    cleantext = BeautifulSoup(str_cells, "lxml").get_text()
    cleantext=cleantext[1:-2]
    column_names=cleantext.split(', ')
    toronto = toronto.append({'Postcode': column_names[0],
                              'Borough': column_names[1],
                              'Neighbourhood': column_names[2]}, ignore_index=True)

In [11]:
toronto.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [12]:
toronto.tail()

,Postcode,Borough,Neighbourhood
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
287,M9Z,Not assigned,Not assigned


### Cleaning  The Data

In [13]:
toronto.drop(toronto[toronto['Borough']=="Not assigned"].index,axis=0, inplace=True)
toronto

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [14]:
toronto_1=toronto.groupby("Postcode").agg(lambda x:','.join(set(x)))
toronto_1

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Richview Gardens,Kingsview Village,St. Phillip..."


In [15]:
toronto_1.shape

(103, 2)